<a href="https://colab.research.google.com/github/anvarxonmajidov/Aviachipta_narxi_praktikum/blob/anvarxonmajidov%2Fdesktop-tutorial/Aviachipta_narxi_mohirdev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn # scikit-learn kutubxonasi
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
sample = pd.read_csv('sample_solution.csv')
test_data = pd.read_csv('test_data.csv',index_col=0)
train_data = pd.read_csv('train_data.csv',index_col=0)
sample.head()

,id,price
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
x_train = train_data.describe()
x_train

,duration,days_left,price
count,20000.000000,20000.000000,20000.000000
mean,12.177627,25.924150,20960.281700
std,7.157944,13.624874,22775.459535
min,0.830000,1.000000,1105.000000
25%,6.830000,14.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.080000,38.000000,42521.000000
max,38.580000,49.000000,114523.000000


In [ ]:
x_train.corrwith(x_train['price'])

duration    -0.099991
days_left   -0.099758
price        1.000000
dtype: float64

In [ ]:
test_data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [ ]:
print(test_data['flight'].value_counts(),train_data['flight'].value_counts(),end="\n")

UK-826     46
UK-706     46
UK-772     44
UK-860     43
UK-720     42
           ..
6E-7127     1
6E-7292     1
G8-2504     1
G8-506      1
6E-186      1
Name: flight, Length: 941, dtype: int64 UK-706     208
UK-720     190
UK-822     178
UK-836     169
UK-772     169
          ... 
6E-2471      1
6E-821       1
SG-1031      1
6E-759       1
6E-7201      1
Name: flight, Length: 1310, dtype: int64


In [ ]:
train_data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           5000 non-null   object 
 1   flight            5000 non-null   object 
 2   source_city       5000 non-null   object 
 3   departure_time    5000 non-null   object 
 4   stops             5000 non-null   object 
 5   arrival_time      5000 non-null   object 
 6   destination_city  5000 non-null   object 
 7   class             5000 non-null   object 
 8   duration          5000 non-null   float64
 9   days_left         5000 non-null   int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 429.7+ KB


In [ ]:
train_data.notnull().sum()

airline             20000
flight              20000
source_city         20000
departure_time      20000
stops               20000
arrival_time        20000
destination_city    20000
class               20000
duration            20000
days_left           20000
price               20000
dtype: int64

In [ ]:
test_data.isnull().sum()

airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
dtype: int64

In [ ]:
X_train = train_data.drop("price", axis=1)
y = train_data["price"].copy()

X_num = X_train[["duration","days_left"]]

In [ ]:
X_train.drop(['flight'], axis=1, inplace=True)
X_train.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,
1,Vistara,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21
2,SpiceJet,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7
3,Vistara,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5
4,Vistara,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28
5,Indigo,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4


In [ ]:
y.head()

id
1     7212
2     5292
3    60553
4     5760
5    10712
Name: price, dtype: int64

In [ ]:
X_num.head()

,duration,days_left
id,,
1,14.25,21
2,1.75,7
3,9.58,5
4,6.75,28
5,2.00,4


#Sonli ustunlar uchun

In [ ]:
num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('std_scaler', StandardScaler())             
])

#Matnli ustunlar uchun

In [ ]:
num_attribs = list(X_num)
cat_attribs = ['airline','source_city','departure_time','stops','arrival_time','destination_city','class']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [ ]:
num_attribs

['duration', 'days_left']

In [ ]:
cat_attribs

['airline',
 'source_city',
 'departure_time',
 'stops',
 'arrival_time',
 'destination_city',
 'class']

In [ ]:
X_prepared = full_pipeline.fit_transform(X_train)
X_prepared[0:5,:]

<5x37 sparse matrix of type '<class 'numpy.float64'>'
	with 45 stored elements in Compressed Sparse Row format>

In [ ]:
LR_model = LinearRegression()
DR_model  = DecisionTreeRegressor()
RR_model = RandomForestRegressor()

#LinearRegression

In [ ]:
LR_model.fit(X_prepared, y)

LinearRegression()

In [ ]:
# tasodifiy 5 ta qatorni ajratib olamiz
test_set = X_train.sample(500)
test_set.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,
19510,GO_FIRST,Bangalore,Night,one,Morning,Hyderabad,Economy,11.75,33
14581,Vistara,Kolkata,Night,one,Morning,Bangalore,Economy,12.25,11
7571,GO_FIRST,Bangalore,Afternoon,one,Night,Delhi,Economy,8.50,11
8574,Vistara,Delhi,Afternoon,one,Night,Mumbai,Economy,6.00,3
11085,Vistara,Kolkata,Morning,one,Evening,Bangalore,Business,8.58,18


In [ ]:
# yuqoridagi qatorlarga mos keluvchi narxlarni ajratib olamiz (biz aynan shu qiymatlarni bashorat qilishimiz kerak)
test_label = y.loc[test_set.index]
test_label.head()

id
19510     4403
14581    11675
7571      7487
8574     23228
11085    60978
Name: price, dtype: int64

In [ ]:
test_data_prepared = full_pipeline.transform(test_set)
test_data_prepared

<500x37 sparse matrix of type '<class 'numpy.float64'>'
	with 4500 stored elements in Compressed Sparse Row format>

In [ ]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data[:5]

array([ 5128.2440192 , 13337.98296614,  8277.07832812, 11238.30010362,
       57927.71064417])

In [ ]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})

,Prognoz,Real baxosi
id,,
19510,5128.244019,4403
14581,13337.982966,11675
7571,8277.078328,7487
8574,11238.300104,23228
11085,57927.710644,60978
...,...,...
5781,58122.539398,62045
12847,8455.787357,4755
17976,4007.622273,4700


In [ ]:
lin_mse = mean_squared_error(test_label, predicted_data)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

6491.128676453064


In [ ]:
test_data.drop(['flight'], axis=1, inplace=True)

In [ ]:
X_test_prepared = full_pipeline.transform(test_data)
X_test_prepared

<5000x37 sparse matrix of type '<class 'numpy.float64'>'
	with 45000 stored elements in Compressed Sparse Row format>

In [ ]:
LR_predicted = LR_model.predict(X_test_prepared)
LR_predicted[:10]

array([55954.43505503, 53515.81514404, 40638.63157655,  2402.92368393,
        5894.53415653,  3743.8730328 ,  8829.6849952 , 12767.84369888,
        6600.11231851, 13803.00109876])

#DecisionTreeRegressor

In [ ]:
DR_model.fit(X_prepared, y)

DecisionTreeRegressor()

In [ ]:
DR_predicted = DR_model.predict(X_test_prepared)
DR_predicted[:10]

array([54863., 51384., 23484.,  2228.,  5102.,  6578.,  4680., 14265.,
        4338., 22366.])

In [ ]:
# tasodifiy 5 ta qatorni ajratib olamiz
test_set = X_train.sample(500)
test_set.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,
9177,Vistara,Delhi,Morning,one,Early_Morning,Hyderabad,Economy,21.50,35
13891,Vistara,Bangalore,Early_Morning,one,Night,Mumbai,Business,15.83,21
10006,Air_India,Hyderabad,Morning,one,Evening,Bangalore,Business,10.08,33
7181,Air_India,Kolkata,Early_Morning,one,Evening,Hyderabad,Economy,12.33,24
12669,Air_India,Mumbai,Afternoon,one,Early_Morning,Chennai,Economy,16.00,40


In [ ]:
# yuqoridagi qatorlarga mos keluvchi narxlarni ajratib olamiz (biz aynan shu qiymatlarni bashorat qilishimiz kerak)
test_label = y.loc[test_set.index]
test_label.head()

id
9177      5772
13891    67004
10006    38043
7181      6062
12669     5380
Name: price, dtype: int64

In [ ]:
test_data_prepared = full_pipeline.transform(test_set)
test_data_prepared

<500x37 sparse matrix of type '<class 'numpy.float64'>'
	with 4500 stored elements in Compressed Sparse Row format>

In [ ]:
predicted_data = DR_model.predict(test_data_prepared)
predicted_data[:5]

array([ 5772., 67004., 38043.,  6062.,  5380.])

In [ ]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})

,Prognoz,Real baxosi
id,,
9177,5772.0,5772
13891,67004.0,67004
10006,38043.0,38043
7181,6062.0,6062
12669,5380.0,5380
...,...,...
1369,55502.0,55502
12259,1714.0,1714
10260,45995.0,45995


In [ ]:
lin_mse = mean_squared_error(test_label, predicted_data)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

194.3366679245067


#RandomForestRegressor

In [ ]:
RR_model.fit(X_prepared, y)

RandomForestRegressor()

In [ ]:
RR_predicted = RR_model.predict(X_test_prepared)
RR_predicted[:10]

array([51432.33  , 55576.46  , 23411.42  ,  2627.54  ,  5663.28  ,
        6449.61  ,  4192.5295, 14033.98  ,  4761.49  , 20952.88  ])

In [ ]:
# tasodifiy 5 ta qatorni ajratib olamiz
test_set = X_train.sample(500)
test_set.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,
13659,Vistara,Bangalore,Morning,one,Evening,Delhi,Economy,8.42,49
15487,Indigo,Hyderabad,Evening,one,Late_Night,Kolkata,Economy,6.92,14
16057,Air_India,Hyderabad,Night,one,Morning,Chennai,Economy,12.08,24
12154,Air_India,Chennai,Early_Morning,one,Morning,Mumbai,Economy,28.00,33
1359,GO_FIRST,Bangalore,Early_Morning,one,Night,Kolkata,Economy,14.42,15


In [ ]:
# yuqoridagi qatorlarga mos keluvchi narxlarni ajratib olamiz (biz aynan shu qiymatlarni bashorat qilishimiz kerak)
test_label = y.loc[test_set.index]
test_label.head()

id
13659    5407
15487    5293
16057    4229
12154    5092
1359     5177
Name: price, dtype: int64

In [ ]:
test_data_prepared = full_pipeline.transform(test_set)
test_data_prepared

<500x37 sparse matrix of type '<class 'numpy.float64'>'
	with 4500 stored elements in Compressed Sparse Row format>

In [ ]:
predicted_data = RR_model.predict(test_data_prepared)
predicted_data[:5]

array([5147.95, 6036.22, 4449.13, 4844.05, 5835.72])

In [ ]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})

,Prognoz,Real baxosi
id,,
13659,5147.95,5407
15487,6036.22,5293
16057,4449.13,4229
12154,4844.05,5092
1359,5835.72,5177
...,...,...
11362,5041.50,5107
16669,3031.94,3014
19989,36204.18,36700


In [ ]:
lin_mse = mean_squared_error(test_label, predicted_data)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

1255.5435660772462


In [ ]:
sample['price'] = RR_predicted
sample

,id,price
0,1,51432.33
1,2,55576.46
2,3,23411.42
3,4,2627.54
4,5,5663.28
...,...,...
4995,4996,59722.18
4996,4997,4387.10
4997,4998,50235.10
4998,4999,48591.00


In [ ]:
sample.price = RR_predicted
sample.to_csv("submission.csv", index=False)
sample

,id,price
0,1,51432.33
1,2,55576.46
2,3,23411.42
3,4,2627.54
4,5,5663.28
...,...,...
4995,4996,59722.18
4996,4997,4387.10
4997,4998,50235.10
4998,4999,48591.00


In [ ]:
pd.read_csv('/content/natija_1.csv')

,Unnamed: 0,id,price
0,0,1,51432.33
1,1,2,55576.46
2,2,3,23411.42
3,3,4,2627.54
4,4,5,5663.28
...,...,...,...
4995,4995,4996,59722.18
4996,4996,4997,4387.10
4997,4997,4998,50235.10
4998,4998,4999,48591.00
